In [29]:
import requests
import json
import mysql.connector

In [30]:
# Finding the field name for "Machine Learning"

header={'Ocp-Apim-Subscription-Key': '9902b9527991400e9c8a47080a515955'}
payload = {'query':"machine learning",'model': 'latest','count': '10','offset': '0','attributes': 'Id'}
r = requests.get('https://api.labs.cognitive.microsoft.com/academic/v1.0/interpret?',params=payload,headers=header)
#print(r.text)
r_dict=r.json()

#print(r_dict)
r_list=r_dict['interpretations']
#print('the interpretations:', r_list[0])
r_interpretRes=r_list[0]['rules'][0]['output']['value']
print(r_interpretRes)


Composite(F.FN=='machine learning')


In [31]:
#Load all the papers and their ID from the above field

payload = {'query':"machine learning",'model': 'latest','count': '10','offset': '0','attributes': 'Id'}
r = requests.get('https://api.labs.cognitive.microsoft.com/academic/v1.0/interpret?',params=payload,headers=header)


In [32]:
con=mysql.connector.connect(user="root",password="root",host="localhost",database="MAG")

In [33]:
mycursor=con.cursor()
mycursor.execute("show tables;");
print(mycursor.fetchall())

[('affiliations',), ('authors',), ('conferences',), ('demo',), ('fieldofstudy',), ('journals',), ('paperauthoraffiliations',), ('papers',)]


In [38]:
"""
https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate?expr=Composite(F.FN=='machine learning')&count=4&attributes= Id,Ti,Y,C.CId,C.CN,J.JId,CC,F.FId,F.FN,AA.AfId,AA.DAuN,AA.AFId,AA.DAfN,AA.AuId
"""

payload = {'expr':'Composite(F.FN==\'machine learning\')','model': 'latest','count': '50','offset': '0','attributes': 'Id,Ti,Y,C.CId,C.CN,J.JId,J.JN,CC,F.FId,F.FN,AA.AfId,AA.DAuN,AA.AFId,AA.DAfN,AA.AuId'}
r = requests.get('https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate?',params=payload,headers=header)
#print(r.text)
r_dict=r.json()
#print(r_dict)

In [39]:
r_entities=r_dict['entities']
print(len(r_entities))

50


In [40]:
for entry in range(len(r_entities)):
    #print(r_entities[entry]['Ti'])
    # collecting paper attributes into the list in the order Id,Ti,Y
    #print(r_dict['entities'][0]['Id'])
    authors=[]
    authors.append(r_entities[entry]['Id'])
    authors.append(r_entities[entry]['Ti'])
    authors.append(r_entities[entry]['CC'])
    authors.append(r_entities[entry]['Y'])
    
    #print(authors)
    #insert into authors
    for i in range(len(r_entities[entry]['AA'])):
        if 'AuId' and 'AfId' in r_entities[entry]['AA'][i]:
            mycursor.execute(f"select * from authors where authorId={r_entities[entry]['AA'][i]['AuId']}")
            count1= mycursor.fetchall()
            print("count1",i)
            print("index ",entry, " ",r_entities[entry]['AA'][i]['AfId'])
            mycursor.execute(f"select * from affiliations where affiliationId={r_entities[entry]['AA'][i]['AfId']}")
            count2= mycursor.fetchall()
         
        #print(count1, count2, len(count1),len(count2))
            if len(count1)==0:
                print("inside if")
                authName=r_entities[entry]['AA'][i]['DAuN'].replace("'","''")
                citationCount=findMissingValues('author',str(r_entities[entry]['AA'][i]['AuId']))
                mycursor.execute(f"insert ignore into authors(authorId,displayName) values({r_entities[entry]['AA'][i]['AuId']},'{authName}')")
            if len(count2)==0:
                afflName=r_entities[entry]['AA'][i]['DAfN'].replace("'","''")
            
                mycursor.execute(f"insert ignore into affiliations(affiliationId,displayName) values({r_entities[entry]['AA'][i]['AfId']},'{afflName}')")
            mycursor.execute(f"insert ignore into paperAuthorAffiliations(paperId,authorId,affiliationId) values({authors[0]},{r_entities[entry]['AA'][i]['AuId']},{r_entities[entry]['AA'][i]['AfId']})")
        #r_entities['AA'][i]['DAuN']
        mycursor.execute("commit")
                             
    print("Inserting into field of study table \n")
    fos=r_entities[entry]['F']
    for i in range(len(fos)):
    #print(i)
        mycursor.execute(f"select * from fieldOfStudy where paperId={authors[0]} and fieldId={fos[i]['FId']}")
        count1=mycursor.fetchall()
    #count1=0;
        if len(count1)==0:
            mycursor.execute(f"insert into fieldOfStudy(fieldId,paperId,displayName) values({fos[i]['FId']},{authors[0]},'{fos[i]['FN']}')")
    mycursor.execute("commit;")

                         
    field=""
    value=""
    safe='no'
    if 'J' in r_entities[entry]:
        journal=r_entities[entry]['J']
        field="journalId"
        print(field)
        print(journal,len(journal))
        value=journal['JId']
        if 'JN' in journal:
            mycursor.execute(f" insert ignore into journals(journalID,displayName) values({value},'{journal['JN']}')")
            mycursor.execute("commit;")
            safe='yes'
        else:
            name=findMissingValues('Journal',value)
            if name !=-999:
                mycursor.execute(f" insert ignore into journals(journalID,displayName) values({idj},'{name}')")
                mycursor.execute("commit;")
                safe='yes'
                    
    elif 'C' in r_entities[entry]:
        conf=r_entities[entry]['C']
        field="conferenceId"
        value=conf['CId']
        print(conf, len(conf))
        if 'CN' in conf:
            mycursor.execute(f" insert ignore into conferences(conferenceID,displayName) values({conf['CId']},'{conf['CN']}')")
            mycursor.execute("commit;") 
            safe='yes'
        else:
            name=findMissingValues('conference',value)
            if name !=-999:
                mycursor.execute(f" insert ignore into conferences(conferenceID,displayName) values({conf['CId']},'{name}')")
                mycursor.execute("commit;") 
                safe='yes'
                 
    if safe=='yes':
        print("Paper "+str(entry))
        mycursor.execute(f" insert ignore into papers(paperId,title,citationCount,publishedDate,{field}) values({authors[0]},'{authors[1]}',{authors[2]},{authors[3]},{value})")
        mycursor.execute("commit;")


count1 0
index  0   16733864
count1 1
index  0   16733864
Inserting into field of study table 

journalId
{'JN': 'acm t intel syst tec', 'JId': 2492086750} 2
Paper 0
count1 0
index  1   141945490
Inserting into field of study table 

journalId
{'JN': 'ijcv', 'JId': 25538012} 2
Paper 1
count1 0
index  2   40120149
count1 1
index  2   40120149
Inserting into field of study table 

{'CN': 'iclr', 'CId': 2584161585} 2
Paper 2
count1 0
index  3   1290206253
count1 1
index  3   87445476
count1 2
index  3   126520041
count1 3
index  3   1290206253
Inserting into field of study table 

{'CN': 'cvpr', 'CId': 1158167855} 2
Paper 3
count1 0
index  4   185261750
count1 1
index  4   185261750
count1 2
index  4   185261750
Inserting into field of study table 

{'CN': 'neurips', 'CId': 1127325140} 2
Paper 4
count1 0
index  5   887064364
count1 1
index  5   185261750
Inserting into field of study table 

{'CN': 'iclr', 'CId': 2584161585} 2
Paper 5
count1 0
index  6   1291425158
count1 1
index  6   114

count1 4
index  49   40120149
Inserting into field of study table 

journalId
{'JN': 'ijcv', 'JId': 25538012} 2
Paper 49


In [ ]:
mycursor.execute("delete from fieldOfStudy")
mycursor.execute("commit")

In [ ]:
ide = 'hello'+str(1)
print(ide)

In [ ]:
if 'AuId' and 'AfId' and 'FN' in r_entities[0]['AA'][0]:
    print("exists")

In [ ]:
def findMissingValues(type,id):
    if type == 'Journal':
        attr='JN'
    elif type=='conference':
        attr='CN'
    elif type=='author':
        attr='CC'
        #print('This is author')
    #print(attr)
    param='Id='+str(id)
    #print(param)
    #header={'Ocp-Apim-Subscription-Key': '9902b9527991400e9c8a47080a515955'}
    payload = {'expr':param,'model': 'latest','attributes': attr}
    r = requests.get('https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate?',params=payload,headers=header)
    r_dict=r.json()
    if 'entities' in r_dict:
        if attr in r_dict['entities'][0]:
            val=r_dict['entities'][0][attr]
            #print(val)
            return val
        else:
            return -999;

if findMissingValues('author','2272624714')!=-999:
    print(findMissingValues('author','2272624714'))


    